 **Routing in LangChain Expression Language**:
  - The concept of routing is explored in LangChain Expression Language, allowing for the creation of non-deterministic chains.
  - In these chains, the output of one step determines the subsequent step, providing a structured and consistent approach to interactions with LLMs.

- **Methods for Implementing Routing**:
  1. **Using RunnableBranch**: This method involves utilizing a `RunnableBranch` for routing purposes.
  2. **Custom Factory Function**: Alternatively, routing can be achieved by writing a custom factory function. This function should take the output of a previous step and return a runnable without executing it.

- **Practical Example of Routing**:
  - An example showcases a two-step sequence:
    - The first step involves classifying an input question as related to LangChain, Anthropic, or Other.
    - Based on this classification, the sequence is then routed to a corresponding prompt chain tailored to the identified category.


### Using a `RunnableBranch`

**Concept of RunnableBranch in LangChain**:
  - A `RunnableBranch` is initialized with a list of pairs, each consisting of a condition and a corresponding runnable, along with a default runnable.
  - The selection of the branch is based on evaluating each condition with the given input.
  - The first condition that evaluates to True triggers the execution of its associated runnable.
  - If none of the provided conditions match, the default runnable is executed.

**Chain Creation and Functionality**

- Different types of chains (Classification, LangChain, Anthropic, General) were created using `PromptTemplate`, `ChatOpenAI`, and `StrOutputParser`.
- Each chain is designed to handle specific types of questions or topics.

**Branching Logic with RunnableBranch**

- The `RunnableBranch` uses lambda functions to route input to the appropriate chain based on topic classification.

**Full Chain Composition and Invocation**

- The `full_chain` combines the classification chain with branching logic.
- The `invoke` method is used to pass a question to the chain for processing and response.

**Simplification with RunnablePassthrough**

- `RunnablePassthrough` was suggested to streamline the chain by passing the entire input dictionary directly to the next step.

**Role of Lambda Functions**

- Lambda functions act like callback functions, executing at the right moment in the chain’s execution to process input.
- They are necessary for dynamic handling of input within the chain.

**Passing Dictionaries in LangChain**

- In LangChain chains, dictionaries are used to pass data between different components of the chain. Each component may modify or add to this dictionary.

**Function of Lambda Functions**

- Lambda functions (`lambda x: ...`) are used to manipulate or access data within these dictionaries. They act as callback functions, executed at specific points in the chain.
- For example, `lambda x: x["question"]` extracts the `"question"` value from the input dictionary.

**Use of RunnablePassthrough**

- `RunnablePassthrough` is a simplification tool that forwards the entire input dictionary to the next component without any modification.
- It's useful when the entire context needs to be passed along the chain.

**Invoking Chains with Dictionaries**

- When invoking a chain (e.g., `full_chain.invoke(...)`), the input is typically a dictionary (e.g., `{"question": "how do i use langchain?"}`).
- This dictionary becomes the input for the first component of the chain.

**Branching Logic and Dictionary Access**

- In branching logic (like `RunnableBranch`), lambda functions check specific keys (like `"topic"`) in the input dictionary to decide the flow of execution.
- The dictionary structure allows for flexible and dynamic handling of input based on the conditions defined in these lambda functions.


In [2]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about
        `LangChain`, `Anthropic`, or `Other`.
        
        Do not respond with more than one word.
        
        <question> {question} </question>
        
        Classification:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)

In [4]:
chain.invoke(
  {
    "question": "how do i call Antrhopic?"
  }
)

'Anthropic'

In [5]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
          Always answer questions starting with "As Harrison Chase told me". \
          Respond to the following question:
        
          Question: {question} Answer:"""
    )
    | ChatOpenAI()
)

anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
          Always answer questions starting with "As Dario Amodei told me". \
          Respond to the following question:
        
          Question: {question} Answer:"""
    )
    | ChatOpenAI()
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:
        
          Question: {question} Answer:"""
    )
    | ChatOpenAI()
)

In [21]:
from langchain_core.runnables import RunnableBranch
from langchain_core.runnables import RunnablePassthrough

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain
  )

In [25]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [26]:
full_chain.invoke(
  {
    "question": "how do i use langchain?"
  }
)

AIMessage(content='As Harrison Chase told me, to use Langchain, you first need to create an account on their website. Once you have an account, you can access their platform and start using their language learning features. They offer various resources such as interactive lessons, vocabulary exercises, and language exchange opportunities. You can track your progress, set goals, and even connect with language tutors for personalized guidance. Langchain also provides a community forum where you can interact with other language learners and seek help when needed. So, start by signing up on their website and explore the different tools and features they offer to enhance your language learning journey.')

In [27]:
full_chain.invoke(
  {
    "question": "how do i use anthropic?"
  }
)

AIMessage(content="As Dario Amodei told me, anthropic is a framework for understanding and reasoning about artificial general intelligence (AGI) and its impact on the world. To use anthropic, you can start by familiarizing yourself with its principles and theories, which aim to address AGI's potential risks and align it with human values. You can study the various papers and writings by Dario Amodei and his colleagues to gain a deeper understanding. Additionally, engaging in discussions and collaborating with the anthropic community can provide valuable insights and perspectives.")

In [28]:
full_chain.invoke(
  {
    "question": "whats 2 + 2?"
  }
)

AIMessage(content='The answer to 2 + 2 is 4.')

### Using a custom function

You can also use a custom function to route between different outputs. Here’s an example:

In [ ]:
def route(info):
  if "anthropic" in info["topic"].lower():
    return anthropic_chain
  elif "langchain" in info["topic"].lower():
    return langchain_chain
  else:
    return general_chain

In [30]:
from langchain_core.runnables import RunnableLambda

full_chain = {
  "topic": chain,
  "question": lambda x: x["question"]
}
| RunnableLambda(route)

SyntaxError: invalid syntax (612748699.py, line 7)